In [1]:
pip install xgboost==1.7.6


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error, r2_score
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import xgboost as xgb

In [4]:
print(xgb.__version__)

1.7.6


In [402]:
all_data = pd.read_csv("model_data.csv",index_col=[0,1],header=[0,1,2])

In [403]:
all_data.index.rename(['track_id', 'genre_top'], inplace=True)

In [404]:
all_data.reset_index(1,inplace=True)

In [405]:
all_data['genre_top'].value_counts()

genre_top
Experimental           24461
Rock                   23084
Electronic             21794
Hip-Hop                 6465
Pop                     6425
Folk                    6170
Instrumental            5690
International           2690
Classical               1675
Spoken                  1335
Country                 1207
Soul-RnB                 941
Jazz                     841
Old-Time / Historic      658
Easy Listening           460
Blues                    447
Name: count, dtype: int64

In [407]:
all_data.dropna(inplace=True)

In [506]:
all_data

genre_top chroma_cens                                          \
                      kurtosis                                           
                            01        02        03        04        05   
track_id                                                                 
2          Hip-Hop    7.180653  5.230309  0.249321  1.347620  1.482478   
3          Hip-Hop    1.888963  0.760539  0.345297  2.295201  1.654031   
5          Hip-Hop    0.527563 -0.077654 -0.279610  0.685883  1.937570   
10             Pop    3.702245 -0.291193  2.196742 -0.234449  1.367364   
20             Pop   -0.193837 -0.198527  0.201546  0.258556  0.775204   
...            ...         ...       ...       ...       ...       ...   
155316        Rock   -0.490129  0.463834  2.321970 -0.084352  1.662914   
155317        Rock   -0.461559 -0.229601 -0.496632 -0.422033  0.130612   
155318        Rock    0.552473 -0.110498 -0.532014  0.263131 -0.224011   
155319        Rock   -0.176901  0.187208 -0.050664  0.368843  0.066005   
155320     Country    0.489665  1.862421  0.854461 -0.103666 -0.249835   

                                                  ...  mel_spec            \
                                                  ...       std             
                06        07        08        09  ...        90        91   
track_id                                          ...                       
2         0.531371  1.481593  2.691455  0.866868  ...  0.005500  0.004066   
3         0.067592  1.366848  1.054094  0.108103  ...  0.014708  0.019320   
5         0.880839 -0.923192 -0.927232  0.666617  ...  0.000010  0.000009   
10        0.998411  1.770694  1.604566  0.521217  ...  0.018939  0.015547   
20        0.084794 -0.289294 -0.816410  0.043851  ...  0.008917  0.008339   
...            ...       ...       ...       ...  ...       ...       ...   
155316    2.115189 -0.237794  5.695442  0.830353  ...  0.001007  0.000663   
155317   -0.263825 -0.628103 -0.082687 -0.229483  ...  0.002338  0.001594   
155318   -0.530972  1.713526  1.418444  1.325197  ...  0.003115  0.002223   
155319   -0.857354 -0.780860  0.626281 -0.630938  ...  0.002461  0.001665   
155320    0.360283 -0.366701  0.033578 -0.834606  ...  0.015526  0.013577   

                                                                      \
                                                                       
                92        93        94        95        96        97   
track_id                                                               
2         0.003965  0.002347  0.001663  0.002632  0.001452  0.000114   
3         0.078528  0.072792  0.069122  0.013616  0.008447  0.001303   
5         0.000008  0.000006  0.000005  0.000007  0.000004  0.000002   
10        0.010535  0.006567  0.006368  0.003965  0.003788  0.002343   
20        0.006849  0.006172  0.005579  0.004760  0.004246  0.004115   
...            ...       ...       ...       ...       ...       ...   
155316    0.000429  0.000345  0.000208  0.000123  0.000121  0.000038   
155317    0.001051  0.000818  0.000603  0.000557  0.000391  0.000214   
155318    0.001846  0.001351  0.000899  0.000688  0.000458  0.000194   
155319    0.001135  0.000730  0.000473  0.000313  0.000198  0.000103   
155320    0.011346  0.010214  0.008648  0.007865  0.006310  0.005185   

                              
                              
                98        99  
track_id                      
2         0.000002  0.000002  
3         0.000003  0.000002  
5         0.000002  0.000001  
10        0.001213  0.000316  
20        0.003781  0.003704  
...            ...       ...  
155316    0.000002  0.000001  
155317    0.000067  0.000098  
155318    0.000042  0.000019  
155319    0.000006  0.000005  
155320    0.003995  0.003733  

[104337 rows x 1415 columns]

In [409]:
new_pop_data = pd.read_csv("new_data_only_pop.csv",header=[0,1,2],index_col = [0,1])

In [410]:
new_pop_data.index.rename(["track_id","genre_top"],inplace=True)

In [411]:
new_pop_data.reset_index([1],inplace=True)

In [412]:
concat_all_data = pd.concat([all_data,new_pop_data])

In [423]:
selected_model_data = all_data[all_data['genre_top'].isin(["Rock","Electronic","Pop","Hip-Hop","Folk"])]

In [425]:
selected_model_data['genre_top'].value_counts()

genre_top
Rock          23083
Electronic    21791
Hip-Hop        6465
Pop            6425
Folk           6170
Name: count, dtype: int64

In [427]:
# selected_model_data.to_csv("final_selected_model_data.csv")

In [428]:
dictG = {"Rock":0,"Electronic":1,"Pop":2,"Hip-Hop":3,"Folk":4}

In [430]:
selected_model_data

genre_top chroma_cens                                          \
                      kurtosis                                           
                            01        02        03        04        05   
track_id                                                                 
2          Hip-Hop    7.180653  5.230309  0.249321  1.347620  1.482478   
3          Hip-Hop    1.888963  0.760539  0.345297  2.295201  1.654031   
5          Hip-Hop    0.527563 -0.077654 -0.279610  0.685883  1.937570   
10             Pop    3.702245 -0.291193  2.196742 -0.234449  1.367364   
20             Pop   -0.193837 -0.198527  0.201546  0.258556  0.775204   
...            ...         ...       ...       ...       ...       ...   
155315        Rock   -0.319850  0.653044  1.300360 -0.606061 -1.109302   
155316        Rock   -0.490129  0.463834  2.321970 -0.084352  1.662914   
155317        Rock   -0.461559 -0.229601 -0.496632 -0.422033  0.130612   
155318        Rock    0.552473 -0.110498 -0.532014  0.263131 -0.224011   
155319        Rock   -0.176901  0.187208 -0.050664  0.368843  0.066005   

                                                  ...  mel_spec            \
                                                  ...       std             
                06        07        08        09  ...        90        91   
track_id                                          ...                       
2         0.531371  1.481593  2.691455  0.866868  ...  0.005500  0.004066   
3         0.067592  1.366848  1.054094  0.108103  ...  0.014708  0.019320   
5         0.880839 -0.923192 -0.927232  0.666617  ...  0.000010  0.000009   
10        0.998411  1.770694  1.604566  0.521217  ...  0.018939  0.015547   
20        0.084794 -0.289294 -0.816410  0.043851  ...  0.008917  0.008339   
...            ...       ...       ...       ...  ...       ...       ...   
155315   -1.006683  0.059643  0.472580 -0.856436  ...  0.000922  0.000430   
155316    2.115189 -0.237794  5.695442  0.830353  ...  0.001007  0.000663   
155317   -0.263825 -0.628103 -0.082687 -0.229483  ...  0.002338  0.001594   
155318   -0.530972  1.713526  1.418444  1.325197  ...  0.003115  0.002223   
155319   -0.857354 -0.780860  0.626281 -0.630938  ...  0.002461  0.001665   

                                                                      \
                                                                       
                92        93        94        95        96        97   
track_id                                                               
2         0.003965  0.002347  0.001663  0.002632  0.001452  0.000114   
3         0.078528  0.072792  0.069122  0.013616  0.008447  0.001303   
5         0.000008  0.000006  0.000005  0.000007  0.000004  0.000002   
10        0.010535  0.006567  0.006368  0.003965  0.003788  0.002343   
20        0.006849  0.006172  0.005579  0.004760  0.004246  0.004115   
...            ...       ...       ...       ...       ...       ...   
155315    0.000138  0.000019  0.000003  0.000004  0.000004  0.000003   
155316    0.000429  0.000345  0.000208  0.000123  0.000121  0.000038   
155317    0.001051  0.000818  0.000603  0.000557  0.000391  0.000214   
155318    0.001846  0.001351  0.000899  0.000688  0.000458  0.000194   
155319    0.001135  0.000730  0.000473  0.000313  0.000198  0.000103   

                              
                              
                98        99  
track_id                      
2         0.000002  0.000002  
3         0.000003  0.000002  
5         0.000002  0.000001  
10        0.001213  0.000316  
20        0.003781  0.003704  
...            ...       ...  
155315    0.000003  0.000002  
155316    0.000002  0.000001  
155317    0.000067  0.000098  
155318    0.000042  0.000019  
155319    0.000006  0.000005  

[63934 rows x 1415 columns]

In [436]:
X = selected_model_data.drop(["genre_top","mel_spec"],axis=1)
y = selected_model_data['genre_top'].replace(dictG)

/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_37014/3181964305.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = selected_model_data.drop(["genre_top","mel_spec"],axis=1)
/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_37014/3181964305.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = selected_model_data['genre_top'].replace(dictG)


In [358]:
# selected_model_data.drop(["genre_top"],axis=1).to_csv("X_features_all.csv")

In [291]:
X.shape

(63934, 518)

In [293]:
y.shape

(63934,)

In [295]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA



# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)


# k = 6
# kmeans = KMeans(n_clusters=k, random_state=42)
# labels = kmeans.fit_predict(X_scaled)

# df = pd.DataFrame()
# df['cluster'] = labels
# df['cluster'] = df[df['cluster'].isin([0,1,2,3,5,6])]['cluster']

# # Step 4: Visualize with PCA (2D projection)
# pca = PCA(n_components=2)
# X_pca = pca.fit_transform(X_scaled)
# X_pca_df = pd.DataFrame(X_pca,columns=["0","1"])
# X_pca_df = X_pca_df[X_pca_df["1"]<1000]
# plt.figure(figsize=(8, 6))
# scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='tab10')
# plt.title(f'KMeans Clustering of Songs (k={k})')
# plt.xlabel('PCA 1')
# plt.ylabel('PCA 2')
# plt.legend(*scatter.legend_elements(), title="Cluster")
# plt.grid(True)
# plt.show()


In [297]:
# plt.figure(figsize=(8, 6))
# scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='tab10')
# plt.title(f'KMeans Clustering of Songs (k={k})')
# plt.xlabel('PCA 1')
# plt.ylabel('PCA 2')
# plt.legend(*scatter.legend_elements(), title="Cluster")
# plt.grid(True)
# plt.show()

In [299]:
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler

# # X_scaled: your scaled feature matrix

# inertia = []
# k_range = range(1, 15)

# for k in k_range:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(X_scaled)
#     inertia.append(kmeans.inertia_)

# plt.figure(figsize=(8, 5))
# plt.plot(k_range, inertia, marker='o')
# plt.title('Elbow Method - Optimal k')
# plt.xlabel('Number of Clusters (k)')
# plt.ylabel('Inertia')
# plt.grid(True)
# plt.show()

In [301]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import classification_report, confusion_matrix




In [303]:
from sklearn.preprocessing import StandardScaler
scaler_standard = StandardScaler()
X = scaler_standard.fit_transform(X)

In [499]:
X.shape

(63934, 518)

In [307]:
y.shape

(63934,)

In [439]:
x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3)
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [441]:
# knn = KNeighborsClassifier(n_neighbors=5)  # You can tune k later
# knn.fit(x_train, y_train)


In [444]:
# from sklearn.metrics import accuracy_score

# accuracies = []

# for k in range(1, 21):
#     knn = KNeighborsClassifier(n_neighbors=k)
#     knn.fit(x_train, y_train)
#     y_pred = knn.predict(x_test)
#     acc = accuracy_score(y_test, y_pred)
#     accuracies.append(acc)
#     print(f"k={k}, Accuracy={acc:.3f}")


In [447]:
# y_pred = knn.predict(x_test)

# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))

# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))


In [449]:
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt

# # PCA to 2D
# X_reduced = PCA(n_components=2).fit_transform(X)

# # Predict with KNN
# y_pred = knn.predict(X)

# # Plot
# plt.figure(figsize=(8, 6))
# scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=pd.factorize(y_pred)[0], cmap='tab10', alpha=0.7)
# plt.title("KNN Predicted Clusters in 2D")
# plt.xlabel("PCA 1")
# plt.ylabel("PCA 2")
# plt.legend(*scatter.legend_elements(), title="Predicted Class")
# plt.grid(True)
# plt.show()


In [451]:
y.value_counts()

genre_top
0    23083
1    21791
3     6465
2     6425
4     6170
Name: count, dtype: int64

In [454]:
# y_train_resampled.value_counts()

In [456]:
model = XGBClassifier(subsample= 0.8, reg_lambda=1.5, n_estimators= 500, max_depth= 5, learning_rate= 0.1, gamma= 0, colsample_bytree= 0.7)

In [458]:
# from sklearn.model_selection import RandomizedSearchCV
# param_grid = {
#     'n_estimators': [100, 300, 500],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.7, 0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0],
#     'gamma': [0, 0.1, 0.2],
#     'reg_lambda': [1, 1.5, 2]
# }
# random_search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=param_grid,
#     n_iter=30,            # number of random combinations to try
#     scoring='accuracy',
#     cv=3,
#     verbose=2,
#     random_state=42,
#     n_jobs=-1
# )

# random_search.fit(x_train, y_train)

# print("Best Random Search Parameters:", random_search.best_params_)
# best_xgb_random = random_search.best_estimator_


In [460]:
# best_xgb_random

In [462]:
# y_train_resampled

In [464]:
# y_train.value_counts()

In [466]:
# y_train_resampled = y_train_resampled.replace(dictG)

In [468]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(sampling_strategy='auto', random_state=42)
# x_train, y_train = smote.fit_resample(x_train, y_train)
from sklearn.utils.class_weight import compute_class_weight
# import numpy as np

# # Suppose y_train has classes like [0, 1, 2, 3]
# classes = np.unique(y_train)
# class_weights = compute_class_weight('balanced', classes=classes, y=y_train)

# print(class_weights)  # one weight for each class

# weights = np.array([class_weights[i] for i in y_train])
import numpy as np
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)

# Now create sample weights
sample_weights = np.array([class_weights[label] for label in y_train])

In [470]:
eval_set = [(x_train, y_train), (x_val, y_val)]

In [472]:
model.fit(x_train,y_train,sample_weight=sample_weights,eval_set=[(x_train, y_train),(x_val, y_val)],verbose=True,early_stopping_rounds=10)

/opt/anaconda3/lib/python3.12/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.56165	validation_1-mlogloss:1.56449
[1]	validation_0-mlogloss:1.51958	validation_1-mlogloss:1.52553
[2]	validation_0-mlogloss:1.48325	validation_1-mlogloss:1.49223
[3]	validation_0-mlogloss:1.45029	validation_1-mlogloss:1.46204
[4]	validation_0-mlogloss:1.42042	validation_1-mlogloss:1.43496
[5]	validation_0-mlogloss:1.39327	validation_1-mlogloss:1.41032
[6]	validation_0-mlogloss:1.36883	validation_1-mlogloss:1.38819
[7]	validation_0-mlogloss:1.34595	validation_1-mlogloss:1.36773
[8]	validation_0-mlogloss:1.32567	validation_1-mlogloss:1.35006
[9]	validation_0-mlogloss:1.30631	validation_1-mlogloss:1.33303
[10]	validation_0-mlogloss:1.28841	validation_1-mlogloss:1.31746
[11]	validation_0-mlogloss:1.27191	validation_1-mlogloss:1.30287
[12]	validation_0-mlogloss:1.25586	validation_1-mlogloss:1.28911
[13]	validation_0-mlogloss:1.24168	validation_1-mlogloss:1.27677
[14]	validation_0-mlogloss:1.22788	validation_1-mlogloss:1.26521
[15]	validation_0-mlogloss:1.21509	

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [487]:
y_train_pred=model.predict(x_train)

In [489]:
print(classification_report(y_train_pred, y_train))

              precision    recall  f1-score   support

           0       0.89      0.96      0.92     14891
           1       0.92      0.95      0.93     14809
           2       0.98      0.87      0.92      5137
           3       1.00      0.88      0.94      5075
           4       1.00      0.88      0.93      4841

    accuracy                           0.93     44753
   macro avg       0.96      0.91      0.93     44753
weighted avg       0.93      0.93      0.93     44753



In [491]:
y_val_pred=model.predict(x_val)

In [493]:
print(classification_report(y_val_pred, y_val))

              precision    recall  f1-score   support

           0       0.71      0.79      0.75      3092
           1       0.76      0.71      0.74      3480
           2       0.27      0.31      0.29       826
           3       0.62      0.57      0.59      1078
           4       0.68      0.58      0.63      1114

    accuracy                           0.67      9590
   macro avg       0.61      0.59      0.60      9590
weighted avg       0.68      0.67      0.67      9590



In [495]:
y_pred = model.predict(x_test)

In [497]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.71      0.74      3452
           1       0.72      0.76      0.74      3268
           2       0.32      0.29      0.31       939
           3       0.58      0.62      0.60       982
           4       0.59      0.70      0.64       950

    accuracy                           0.68      9591
   macro avg       0.60      0.62      0.61      9591
weighted avg       0.68      0.68      0.68      9591



In [278]:
import pickle
with open("final_genre_classification_model_71.pkl", 'wb') as file:
        pickle.dump(model, file)

In [108]:
classified_genre = pd.DataFrame(model.predict_proba(X),index=selected_model_data.index,columns=["Rock","Electronic","Pop","Hip-Hop","Folk"])

In [110]:
classified_genre.to_csv("classified_genre_prob.csv")

In [104]:
y

track_id
2         3
3         3
5         3
10        2
20        2
         ..
155315    0
155316    0
155317    0
155318    0
155319    0
Name: genre_top, Length: 63934, dtype: int64

In [161]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=500)  # select top 50 features
X_train_selected = selector.fit_transform(x_train, y_train)
X_test_selected = selector.transform(x_test)
X_val_selected = selector.transform(x_val)

In [163]:
# import matplotlib.pyplot as plt

# # Plot top 50 features
# top_n = 50

# plt.figure(figsize=(10, 12))
# plt.barh(feat_importances['feature'][:top_n], feat_importances['importance'][:top_n])
# plt.gca().invert_yaxis()  # Most important on top
# plt.xlabel('Importance')
# plt.title(f'Top {top_n} Feature Importances')
# plt.show()

In [197]:
model.fit(X_train_selected,y_train,sample_weight=sample_weights,eval_set=[(X_val_selected, y_val)],verbose=True,early_stopping_rounds=10)

/opt/anaconda3/lib/python3.12/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.56194
[1]	validation_0-mlogloss:1.51839
[2]	validation_0-mlogloss:1.48058
[3]	validation_0-mlogloss:1.44643
[4]	validation_0-mlogloss:1.41677
[5]	validation_0-mlogloss:1.39059
[6]	validation_0-mlogloss:1.36603
[7]	validation_0-mlogloss:1.34450
[8]	validation_0-mlogloss:1.32490
[9]	validation_0-mlogloss:1.30644
[10]	validation_0-mlogloss:1.28989
[11]	validation_0-mlogloss:1.27439
[12]	validation_0-mlogloss:1.25897
[13]	validation_0-mlogloss:1.24567
[14]	validation_0-mlogloss:1.23330
[15]	validation_0-mlogloss:1.22200
[16]	validation_0-mlogloss:1.21135
[17]	validation_0-mlogloss:1.20178
[18]	validation_0-mlogloss:1.19207
[19]	validation_0-mlogloss:1.18319
[20]	validation_0-mlogloss:1.17530
[21]	validation_0-mlogloss:1.16789
[22]	validation_0-mlogloss:1.16038
[23]	validation_0-mlogloss:1.15353
[24]	validation_0-mlogloss:1.14642
[25]	validation_0-mlogloss:1.14018
[26]	validation_0-mlogloss:1.13417
[27]	validation_0-mlogloss:1.12804
[28]	validation_0-mlogloss:1.1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [198]:
X_train_selected.shape

(44753, 500)

In [199]:
# y_test_pred = best_xgb_random.predict(X_test_selected)

In [200]:
y_train_pred=model.predict(X_train_selected)

In [201]:
print(classification_report(y_train_pred, y_train))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92     14652
           1       0.91      0.94      0.93     14864
           2       0.98      0.87      0.92      5065
           3       1.00      0.88      0.93      5147
           4       1.00      0.87      0.93      5025

    accuracy                           0.93     44753
   macro avg       0.95      0.90      0.93     44753
weighted avg       0.93      0.93      0.93     44753



In [202]:
y_val_pred=model.predict(X_val_selected)

In [203]:
print(classification_report(y_val_pred, y_val))

              precision    recall  f1-score   support

           0       0.72      0.79      0.75      3206
           1       0.76      0.73      0.74      3403
           2       0.34      0.39      0.36       846
           3       0.63      0.61      0.62      1003
           4       0.73      0.57      0.64      1132

    accuracy                           0.69      9590
   macro avg       0.63      0.62      0.62      9590
weighted avg       0.69      0.69      0.69      9590



In [204]:
y_pred = model.predict(X_test_selected)

In [205]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.74      0.77      3499
           1       0.72      0.76      0.74      3264
           2       0.35      0.31      0.32       969
           3       0.60      0.61      0.60       971
           4       0.56      0.67      0.61       888

    accuracy                           0.68      9591
   macro avg       0.61      0.62      0.61      9591
weighted avg       0.68      0.68      0.68      9591



In [194]:
# print(classification_report(y_test_pred, y_test))

In [176]:
bank_acc = pd.read_csv("bank_acc.csv",header=[0,1,2],index_col=[0])

FileNotFoundError: [Errno 2] No such file or directory: 'bank_acc.csv'

In [272]:
bank_acc

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
0            -0.363607  0.010086 -1.228869 -1.166695  0.106115  1.132824   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
0           2.960157 -0.359578  0.287732  4.898821  ...  0.244532  0.086667   

feature                     zcr                                            \
statistics             kurtosis    max      mean    median  min      skew   
number            06         01     01        01        01   01        01   
0           0.072306  16.283276  0.375  0.029002  0.020508  0.0  2.981179   

feature               
statistics       std  
number            01  
0           0.031351  

[1 rows x 1414 columns]

In [274]:
bank_acc_prob = pd.DataFrame(model.predict_proba(bank_acc),columns=["Rock","Electronic","Pop","Hip-Hop","Folk"])

In [221]:
central_cee_hiphop_prob

,Rock,Electronic,Pop,Hip-Hop,Folk
0,0.651854,0.220059,0.101244,0.011218,0.015625


In [276]:
bank_acc_prob

,Rock,Electronic,Pop,Hip-Hop,Folk
0,0.431281,0.459316,0.091708,0.012794,0.004901


In [147]:
highest_in_the_room_prob.to_csv("highest_in_the_room_prob.csv")

In [10]:
def load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks


In [14]:
features = load("../../fma_metadata/features.csv")

In [15]:
tracks = load("../../fma_metadata/tracks.csv")

In [38]:
tracks['track','favorites']

track_id
2           2
3           1
5           6
10        178
20          0
         ... 
155316      1
155317      1
155318      2
155319      0
155320      1
Name: (track, favorites), Length: 106574, dtype: int64

In [44]:
tracks['track','interest']


track_id
2          4656
3          1470
5          1933
10        54881
20          978
          ...  
155316      122
155317      194
155318      214
155319      336
155320      972
Name: (track, interest), Length: 106574, dtype: int64

In [48]:
tracks.track.columns

Index(['bit_rate', 'comments', 'composer', 'date_created', 'date_recorded',
       'duration', 'favorites', 'genre_top', 'genres', 'genres_all',
       'information', 'interest', 'language_code', 'license', 'listens',
       'lyricist', 'number', 'publisher', 'tags', 'title'],
      dtype='object')